In [1]:
#текущая папка (для колаба)
base = 'drive/My Drive/GPB'

In [2]:
import numpy as np
import pandas as pd
import scipy.stats as st
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')
%config InlineBackend.figure_format = 'retina'
import re
import sys
import datetime

in_colab = 'google.colab' in sys.modules
if in_colab:
    sys.path.append(base)

!sudo pip3 install rutimeparser
import preprocessing

def get_drive_path(path):
    in_colab = 'google.colab' in sys.modules
    return '/content/'+base+'/'+re.sub(r'\\','/',path) if in_colab else path

In [3]:
#кодировки классов (каждому классу соответсвует число)
code_names_path = get_drive_path('data/Классы_кодировка.csv')
code_to_class, class_to_code = preprocessing.get_code_names(code_names_path)

In [4]:
def load_data(path):
    data = pd.read_csv(path)
    data.rename({'Ответ 2 из 3:':target_name}, axis=1, inplace=True)
    data = data[data[target_name] != class_to_code['_неоднозначность']]
    return data

target_name = 'class'

train_path = get_drive_path('data\\train_data_19900_04-02.txt')
test_path = get_drive_path('data\\test_data_19900_04-02.txt')

train_data = load_data(train_path)
test_data = load_data(test_path) 
all_data = pd.concat((train_data, test_data))

In [5]:
!pip install faker
import graph_generation as gg

unique, counts = np.unique(all_data[target_name].values, return_counts=True)
classes_freqs = pd.DataFrame({'class':[code_to_class[class_] for class_ in unique], 
                              'freq':counts})
classes_freqs.to_csv(get_drive_path('data\\graph\\data for graph gen\\class_freqs.csv'))

     |████████████████████████████████| 1.2MB 6.9MB/s 


In [6]:
import importlib
importlib.reload(gg)

np.random.seed(41)

generator = gg.GraphGenerator(folder_path=get_drive_path('data\\graph\\data for graph gen'))
transactions = generator.generate_transactions(N=30000)
transactions

,time,from,to,class,sum
0,2018-01-01 00:03:00,102785297274,620899547631,возврат,2.089481e+05
1,2018-01-01 00:08:00,547816499741,297362516189,страховая премия,4.895826e+06
2,2018-01-01 00:17:00,248871250175,335738562249,возврат,4.356897e+08
3,2018-01-01 00:35:00,453100990156,246732935475,возврат,1.192398e+05
4,2018-01-01 00:54:00,335738562249,856585951301,займ,2.500175e+07
...,...,...,...,...,...
29995,2018-07-28 01:05:00,070338560330,240109134266,комиссии банку,6.636937e+02
29996,2018-07-28 01:13:00,829832223778,891053699888,страховая премия,6.512500e+04
29997,2018-07-28 01:32:00,397082749548,600194594217,возврат,2.398600e+07
29998,2018-07-28 01:35:00,471347271270,375220490043,взыскание с фл,2.993137e+03


In [ ]:
# Балансы компаний, для которых наступил дефолт
for agent in generator.agents_company_dict:
    for inn in generator.agents_company_dict[agent]:
        default_time = generator.agents_company_dict[agent][inn]['default time']
        if default_time is not None:
            print(generator.agents_company_dict[agent][inn]['balance'])

0.0
0.0
0.0
0.0
0.0
0.0
419766281.0738234
549829673.529298
728487725.1074083
764101544.2082077
90880525.35878174
26293858.815148234
419907284.52880335
214136793.0771612
94661707.90939955
1096651196.942166
1007602182.6728234
1145545151.6381636
1244559040.9417708
1010802072.4948219
1008754836.3142182
1022888780.2269139
1097732866.1297662
1005988237.2119051
1011966339.4961232
0.0
0.0
0.0
0.0
0.0
0.0
0.0
791044719.0200491
954155008.2173908
748273048.2169913
839567649.2260514
910909649.5854177
1002386167.3437704
993519126.5914603
1260083349.2685215
991143310.2389508
1000413468.9171413
1008053317.906789
1062180753.0139397
1141957244.0663056
989926306.4937155
1049988234.0091646
0.0
2382738.087832204
1122366.6617528945
0.0
0.0
3620358.807309419
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
16851677.055744562
0.0
0.0
87945937.37175281
15948755.469370428
75049701.19507927
0.0
10323733.53850305
410238319.0673151
752546326.61603
984344612.8921885
2

In [10]:
import pickle
# Сохранение данных в txt
companies_literals_dfs = []
companies_attrs_dfs = []

for i,_ in enumerate(generator.agents_company_dict_list):
    companies_literals = []
    companies_attrs = []
    companies = []
    for agent in generator.agents_company_dict_list[i]:
        for inn in generator.agents_company_dict_list[i][agent]:
            dict_ = generator.agents_company_dict_list[i][agent][inn]

            balance = dict_["balance"]
            default_time = (dict_["default time"].strftime("%m-%d-%Y %H:%M:%S") if
                dict_["default time"] is not None else 'None')

            companies_literals.append([inn, 'время дефолта', default_time])
            companies_literals.append([inn, 'баланс', balance])
            companies_literals.append([inn, 'тип агента', agent])

            companies_attrs.append([inn, agent, balance, default_time])

            companies.append(inn)

    
    companies_literals = pd.DataFrame(companies_literals, columns=['inn', 'attr', 'value'])
    companies_literals_dfs.append(companies_literals)

    companies_attrs = pd.DataFrame(companies_attrs, columns=['inn', 'agent', 'balance', 'default time'])
    companies_attrs_dfs.append(companies_attrs)

    if i == 0:
        pd.DataFrame(companies).to_excel(get_drive_path('data\\graph\\companies.xlsx'))

pd.to_pickle(companies_literals_dfs, get_drive_path('data\\graph\\companies_literals.pkl'))
pd.to_pickle(companies_attrs_dfs, get_drive_path('data\\graph\\companies_attrs.pkl'))
pd.to_pickle(transactions, get_drive_path('data\\graph\\transactions.pkl'))

transactions.to_excel(get_drive_path('data\\graph\\transactions.xlsx'))

FileNotFoundError: ignored